- sales and prices correlation 
- sales on different states 
- sales and prices difference on special events 



In [1]:
import pandas as pd 
import numpy as np 
import gc

In [2]:
def transform_df(): 
    calender_dtype={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "uint8", 'snap_TX': 'uint8', 'snap_WI': 'uint8' }
    calender = pd.read_csv('calendar.csv', dtype= calender_dtype)
    calender.date = pd.to_datetime(calender.date)
         
    price_dtype = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }
    #prices of products 
    product_prices = pd.read_csv('sell_prices.csv', dtype= price_dtype)  
  
    days = 1913
    colstype = ['id', "store_id", 'state_id', 'item_id', 'cat_id', 'dept_id']
    d_numtype = [f"d_{day}" for day in range(1,days+1)]
    
    dtype = {num: 'int8' for num in d_numtype}
    dtype.update({col: "category" for col in colstype}) 
    
    daily_temp = pd.read_csv('sales_train_validation.csv', usecols = d_numtype + colstype, dtype = dtype)
    
    daily_sales = pd.melt(daily_temp,id_vars = ['id', 'store_id', 'item_id','state_id','cat_id', 'dept_id'], 
                      var_name = "d", value_name = 'Sales')
    
    gc.collect()
    
    daily_sales = daily_sales.merge(calender_df, on = 'd', copy = 'false')
    dataset = daily_sales.merge(product_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], copy = False)
    
    dataset = dataset.drop(columns = ['id', 'd', 'cat_id', 'month', 'year', 'weekday', 'wday']) #since they are almost duplicates, we can drop the columns. 
    gc.collect() #free memory 

    return dataset


In [ ]:
dataset = transform_df()

In [21]:
CA_df = dataset[dataset['state_id'] == 'CA']
TX_df = dataset[dataset['state_id'] == 'TX']
WI_df = dataset[dataset['state_id'] == 'WI']

gc.collect()


,store_id,item_id,state_id,dept_id,Sales,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
31605,TX_1,HOBBIES_1_004,TX,HOBBIES_1,1,2011-01-29,11101,NaN,NaN,NaN,NaN,0.0,0.0,0.0,4.34
31606,TX_1,HOBBIES_1_004,TX,HOBBIES_1,2,2011-01-30,11101,NaN,NaN,NaN,NaN,0.0,0.0,0.0,4.34
31607,TX_1,HOBBIES_1_004,TX,HOBBIES_1,1,2011-01-31,11101,NaN,NaN,NaN,NaN,0.0,0.0,0.0,4.34
31608,TX_1,HOBBIES_1_004,TX,HOBBIES_1,0,2011-02-01,11101,NaN,NaN,NaN,NaN,1.0,1.0,0.0,4.34
31609,TX_1,HOBBIES_1_004,TX,HOBBIES_1,2,2011-02-02,11101,NaN,NaN,NaN,NaN,1.0,0.0,1.0,4.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46009658,TX_3,FOODS_3_825,TX,FOODS_3,0,2016-04-24,11613,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.98
46009659,TX_3,FOODS_3_826,TX,FOODS_3,1,2016-04-23,11613,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.28
46009660,TX_3,FOODS_3_826,TX,FOODS_3,2,2016-04-24,11613,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.28
46009661,TX_3,FOODS_3_827,TX,FOODS_3,0,2016-04-23,11613,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.00


In [22]:
CA_df.to_csv('CA_dataset.csv')
WI_df.to_csv('WI_dataset.csv')
TX_df.to_csv('TX_dataset.csv')

,store_id,item_id,state_id,dept_id,Sales,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,CA_1,HOBBIES_1_008,CA,HOBBIES_1,12,2011-01-29,11101,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.46
1,CA_1,HOBBIES_1_008,CA,HOBBIES_1,15,2011-01-30,11101,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.46
2,CA_1,HOBBIES_1_008,CA,HOBBIES_1,0,2011-01-31,11101,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.46
3,CA_1,HOBBIES_1_008,CA,HOBBIES_1,0,2011-02-01,11101,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.46
4,CA_1,HOBBIES_1_008,CA,HOBBIES_1,0,2011-02-02,11101,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45991364,CA_4,FOODS_3_825,CA,FOODS_3,0,2016-04-24,11613,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.98
45991365,CA_4,FOODS_3_826,CA,FOODS_3,0,2016-04-23,11613,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.28
45991366,CA_4,FOODS_3_826,CA,FOODS_3,4,2016-04-24,11613,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.28
45991367,CA_4,FOODS_3_827,CA,FOODS_3,4,2016-04-23,11613,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.00
